## 감성 분석(Scikit-Learn)
---
Scikit-Learn
로지스틱 회귀를 활용하여 감성사전을 만들고 감성분석하기

### 1. 감성 사전 만들기

#### 아마존 제품 리뷰 TDM 분석 결과 및 원본 파일 불러오기

In [1]:
import joblib

In [2]:
with open('amazon.pkl', 'rb') as f:
    data = joblib.load(f)
locals().update(data)

#### 학습용 / 테스트용 데이터 분할

In [3]:
from sklearn.model_selection import train_test_split

In [24]:
X_train, X_test, y_train, y_test = train_test_split(tdm, sentiment, test_size = 0.2, random_state=1234)

#### 모델 생성

In [27]:
from sklearn.linear_model import LogisticRegression

In [28]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [29]:
model.score(X_train, y_train)

0.9425

In [30]:
model.score(X_test, y_test)

0.785

#### 회귀 계수 확인

In [31]:
model.coef_.shape

(1, 1000)

In [32]:
model.coef_[0, :10]

array([ 0.13694391,  0.19307472, -0.12710473, -0.15407137,  0.        ,
        0.10452614, -0.17361094,  0.11934668,  0.        ,  0.        ])

#### 단어와 회귀 계수 프레임 생성

In [33]:
import pandas as pd

In [34]:
sent_df = pd.DataFrame({"단어": vectorizer.get_feature_names(),
                       "계수": model.coef_.flat})

C:\Users\User\anaconda3\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [35]:
sent_df

,단어,계수
0,10,0.136944
1,100,0.193075
2,11,-0.127105
3,12,-0.154071
4,13,0.000000
...,...,...
995,wrongly,-0.127689
996,year,0.000871
997,years,0.799594
998,yell,-0.238324


#### 단어 사전

In [36]:
# 부정 단어 사전 확인
neg_df = sent_df[sent_df["계수"]<0].sort_values(by="계수")
# 긍정 단어 사전 확인
neg_df = sent_df[sent_df["계수"]>0].sort_values(by="계수", ascending=False)

# 감성 사전 저장
sent_df.to_csv('sent_df.csv')

### 2. 감성 사전 활용 감성 점수 확인

#### 감성 사전 불러오기

In [37]:
import pandas as pd

In [38]:
sent_df2 = pd.read_csv('sent_df.csv', index_col=0)

#### 극성 계산

In [39]:
import numpy as np

In [40]:
sent_df2['극성'] = np.sign(sent_df2["계수"])
sent_df2.set_index("단어", inplace=True)

In [41]:
# 테스트 문장
sentence = "poor screen but reasonable price"

In [42]:
words = sentence.split()          #토큰화
sent_df2.reindex(words)

,계수,극성
단어,,
poor,-1.559596,-1.0
screen,0.145757,1.0
but,NaN,NaN
reasonable,0.180332,1.0
price,1.658992,1.0


In [43]:
# 없는 단어 제외
sent = sent_df2.reindex(words).dropna()
# 극성 계산
sent["극성"].sum() / sent["극성"].abs().sum()

0.5